In [1]:
import pandas as pd
from pandas import Series, DataFrame
import country_converter as coco
import os

In [2]:
path = '/Users/ariana/desktop/historical_tech/raw data/irena'
os.chdir(path)

In [3]:
# https://pxweb.irena.org/pxweb/en/IRENASTAT

In [4]:
irena = os.listdir(path)
for file in irena:
    if '.xlsx' not in file:
        irena.remove(file)

In [5]:
def read_irena(file_name):
    file = os.path.abspath(file_name)
    df = pd.read_excel(file, header=2, na_values='..', skipfooter=26)
    df.dropna(how='all', inplace=True)
    df.rename(columns={'Unnamed: 0': 'Country Name'}, inplace=True)
    df['Data Source'] = 'IRENA'
    df['Spatial Scale'] = 'National'
    df['Metric'] = 'Installed electricity capacity'
    df['Unit'] = 'MW'
    iso2 = []
    for country in df['Country Name']:
        iso2.append(coco.convert(names=country, to='iso2'))
    df['Country Code'] = iso2
    tech = df['Unnamed: 1'].iloc[0]
    grid = df['Unnamed: 2'].iloc[0]
    tech_name = grid + ' ' + tech
    df['Technology Name'] = tech_name.title()
    df.drop(columns=['Unnamed: 1', 'Unnamed: 2'], inplace=True)
    df['ID'] = df['Technology Name'] + '_' + df['Metric'] + '_' + df['Country Code']
    df.set_index('ID', inplace=True)
    return df

In [6]:
irena_df = []
for tech in irena:
    df = read_irena(tech)
    irena_df.append(df)

In [7]:
irena_concat = pd.concat(irena_df)

In [8]:
omit = []
for col in irena_concat.columns:
    if len(col) != 4 or col == 'Unit':
        omit.append(col)
empty_rows = irena_concat.drop(columns=omit)
empty_rows.dropna(how='all', inplace=True)
na_idx = []
for country in irena_concat.index:
    if country not in empty_rows.index:
        na_idx.append(country)
irena_concat.drop(na_idx, inplace=True)

In [9]:
irena_concat.to_csv('/Users/ariana/desktop/historical_tech/cleaned data/irena.csv')